In [1]:
import os
import fire
import argparse

import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import (
    Dataset, 
    DataLoader
)

import torch.nn.functional as F
from tqdm import tqdm, trange

from transformers import (
    RobertaTokenizer,
    set_seed
)

# Config

In [2]:
FLAG = False
MIXUP_START = 10
LAMBDA = 0.5

INPUT_COLUMN = 'text'
DATA_COLUMN = 'category'
OUTPUT_COLUMN = 'label'

NUM_EPOCHS = 2
MAX_LEN = 256
BATCH_SIZE = 32

print("INPUT_COLUMN: ", INPUT_COLUMN)
print("DATA_COLUMN: ", DATA_COLUMN)
print("OUTPUT_COLUMN: ", OUTPUT_COLUMN)
print("NUM_EPOCHS: ", NUM_EPOCHS)
print("MAX_LEN: ", MAX_LEN)
print("BATCH_SIZE: ", BATCH_SIZE)
print("LAMBDA: ", LAMBDA)
print("FLAG: ", FLAG)
print("MIXUP_START: ", MIXUP_START)

INPUT_COLUMN:  text
DATA_COLUMN:  category
OUTPUT_COLUMN:  label
NUM_EPOCHS:  2
MAX_LEN:  256
BATCH_SIZE:  32
LAMBDA:  0.5
FLAG:  False
MIXUP_START:  10


# Mixup Roberta

In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Optional, Tuple, Union

from transformers.models.roberta.modeling_roberta import (
    RobertaPreTrainedModel,
    RobertaLayer,
    RobertaEmbeddings,
    RobertaPooler,
    RobertaClassificationHead
)

from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions,
    SequenceClassifierOutput
)

# from config import *


class RobertaMixerEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.layer = nn.ModuleList([RobertaLayer(config) for _ in range(config.num_hidden_layers)])
        self.gradient_checkpointing = False

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = False,
        output_hidden_states: Optional[bool] = False,
        return_dict: Optional[bool] = True,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPastAndCrossAttentions]:
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None

        next_decoder_cache = () if use_cache else None
        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None
            past_key_value = past_key_values[i] if past_key_values is not None else None

            if self.gradient_checkpointing and self.training:

                if use_cache:
                    logger.warning(
                        "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                    )
                    use_cache = False

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        return module(*inputs, past_key_value, output_attentions)

                    return custom_forward

                layer_outputs = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(layer_module),
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                )
            else:
                layer_outputs = layer_module(
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,
                )

            hidden_states = layer_outputs[0]
            if use_cache:
                next_decoder_cache += (layer_outputs[-1],)
            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (layer_outputs[2],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v
                for v in [
                    hidden_states,
                    next_decoder_cache,
                    all_hidden_states,
                    all_self_attentions,
                    all_cross_attentions,
                ]
                if v is not None
            )
        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=next_decoder_cache,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )


class RobertaMixerModel(RobertaPreTrainedModel):
    """
    The model can behave as an encoder (with only self-attention) as well as a decoder, in which case a layer of
    cross-attention is added between the self-attention layers, following the architecture described in *Attention is
    all you need*_ by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz
    Kaiser and Illia Polosukhin.
    To behave as an decoder the model needs to be initialized with the `is_decoder` argument of the configuration set
    to `True`. To be used in a Seq2Seq model, the model needs to initialized with both `is_decoder` argument and
    `add_cross_attention` set to `True`; an `encoder_hidden_states` is then expected as an input to the forward pass.
    .. _*Attention is all you need*: https://arxiv.org/abs/1706.03762
    """

    _keys_to_ignore_on_load_missing = [r"position_ids"]

    # Copied from transformers.models.bert.modeling_bert.BertModel.__init__ with Bert->Roberta
    def __init__(self, config, add_pooling_layer=True):
        super().__init__(config)
        self.config = config

        self.embeddings = RobertaEmbeddings(config)
        self.encoder = RobertaMixerEncoder(config)

        self.pooler = RobertaPooler(config) if add_pooling_layer else None

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value

    def _prune_heads(self, heads_to_prune):
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer} See base
        class PreTrainedModel
        """
        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)


            
    # Copied from transformers.models.bert.modeling_bert.BertModel.forward
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.Tensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPoolingAndCrossAttentions]:
        r"""
        encoder_hidden_states  (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`, *optional*):
            Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention if
            the model is configured as a decoder.
        encoder_attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used in
            the cross-attention if the model is configured as a decoder. Mask values selected in `[0, 1]`:
            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
        past_key_values (`tuple(tuple(torch.FloatTensor))` of length `config.n_layers` with each tuple having 4 tensors of shape `(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
            Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.
            If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
            don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
            `decoder_input_ids` of shape `(batch_size, sequence_length)`.
        use_cache (`bool`, *optional*):
            If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
            `past_key_values`).
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if self.config.is_decoder:
            use_cache = use_cache if use_cache is not None else self.config.use_cache
        else:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        batch_size, seq_length = input_shape
        device = input_ids.device if input_ids is not None else inputs_embeds.device

        # past_key_values_length
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if attention_mask is None:
            attention_mask = torch.ones(((batch_size, seq_length + past_key_values_length)), device=device)

        if token_type_ids is None:
            if hasattr(self.embeddings, "token_type_ids"):
                buffered_token_type_ids = self.embeddings.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape)

        # If a 2D or 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        if not return_dict:
            return (sequence_output, pooled_output) + encoder_outputs[1:]

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values=encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )



class RobertaMixerForSequenceClassification(RobertaPreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"position_ids"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.roberta = RobertaMixerModel(config, add_pooling_layer=False)
        
        if FLAG:
            self.mixup_dense = nn.Linear(config.hidden_size, config.hidden_size)
            self.mixup_layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
            self.mixup_dropout = nn.Dropout(config.hidden_dropout_prob)

        self.classifier = RobertaClassificationHead(config)

        # Initialize weights and apply final processing
        self.post_init()

    
    
    def forward(
        self,
        input_ids_1: Optional[torch.LongTensor] = None,
        attention_mask_1: Optional[torch.FloatTensor] = None,
        input_ids_2: Optional[torch.LongTensor] = None,
        attention_mask_2: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels_1: Optional[torch.LongTensor] = None,
        labels_2: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs_1 = self.roberta(
            input_ids_1,
            attention_mask=attention_mask_1,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output_1 = outputs_1[0]
        
        # Mixup train
        if (input_ids_2 is not None) and (attention_mask_2 is not None) and (labels_2 is not None):
            
            outputs_2 = self.roberta(
                input_ids_2,
                attention_mask=attention_mask_2,
                position_ids=position_ids,
                head_mask=head_mask,
                inputs_embeds=inputs_embeds,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )
            sequence_output_2 = outputs_2[0]

            sequence_output = (LAMBDA * sequence_output_1) + ((1 - LAMBDA) * sequence_output_2)

            if FLAG:
                sequence_output = self.mixup_dense(sequence_output)
                sequence_output = self.mixup_layernorm(sequence_output)
                sequence_output = self.mixup_dropout(sequence_output)

            logits = self.classifier(sequence_output)

            loss = None
            loss_fct = nn.CrossEntropyLoss()
            loss = (LAMBDA * loss_fct(logits.view(-1, self.num_labels), labels_1.view(-1))) + ((1 - LAMBDA) * loss_fct(logits.view(-1, self.num_labels), labels_2.view(-1)))

        # Mixup eval
        else:
            logits = self.classifier(sequence_output_1)
            loss = None
            if labels_1 is not None:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels_1.view(-1))
         
        # Return logits, loss, and hidden states
        if not return_dict:
            output = (logits,) + outputs_1[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs_1.hidden_states,
            attentions=outputs_1.attentions,
        )

## MixupDataset

In [4]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [5]:
# def prepare_dataset(data, info_data=None, mixup=False):
    
#     # Remove none and hard examples
#     data = data[(data['category'] != 'none') & (data['category'] != 'hard')].reset_index(drop=True)

#     # Add softmax and entropy info
#     if (info_data is not None) and mixup:
#         data = pd.merge(data, info_data, on='idx')[['idx', 'text', 'label', 'category', 'softmax', 'entropy']]
#         mixup_size = len(info_data) - len(data)

#         # --------------------------------------- Same class mixup ---------------------------------------  

#         # Easy-Easy Mixup
#         easy_data = data[data['category'] == 'easy']
#         easy_low_ent_idx = easy_data.sort_values('entropy', ascending=True).head(mixup_size//3)['idx'].tolist()
#         easy_high_ent_idx = easy_data.sort_values('entropy', ascending=False).head(mixup_size//3)['idx'].tolist()
        
#         easy_mixup_data = easy_data[easy_data['idx'].isin(easy_low_ent_idx)].reset_index(drop=True)
#         random.shuffle(easy_high_ent_idx)
#         easy_mixup_data['idx_2'] = easy_high_ent_idx
#         easy_mixup_data['text_2'] = easy_mixup_data['idx_2'].apply(lambda x: easy_data[easy_data['idx'] == x]['text'].values[0])
#         easy_mixup_data['label_2'] = easy_mixup_data['idx_2'].apply(lambda x: easy_data[easy_data['idx'] == x]['label'].values[0])
#         easy_mixup_data['category_2'] = easy_mixup_data['idx_2'].apply(lambda x: easy_data[easy_data['idx'] == x]['category'].values[0])
#         easy_mixup_data['mixup_type'] = 'same_easy'
        
#         # Ambi-Ambi Mixup
#         ambiguous_data = data[data['category'] == 'ambiguous']
#         ambiguous_low_ent_idx = ambiguous_data.sort_values('entropy', ascending=True).head(mixup_size//3)['idx'].tolist()
#         ambiguous_high_ent_idx = ambiguous_data.sort_values('entropy', ascending=False).head(mixup_size//3)['idx'].tolist()
        
#         ambiguous_mixup_data = ambiguous_data[ambiguous_data['idx'].isin(ambiguous_low_ent_idx)].reset_index(drop=True)
#         random.shuffle(ambiguous_high_ent_idx)
#         ambiguous_mixup_data['idx_2'] = ambiguous_high_ent_idx
#         ambiguous_mixup_data['text_2'] = ambiguous_mixup_data['idx_2'].apply(lambda x: ambiguous_data[ambiguous_data['idx'] == x]['text'].values[0])
#         ambiguous_mixup_data['label_2'] = ambiguous_mixup_data['idx_2'].apply(lambda x: ambiguous_data[ambiguous_data['idx'] == x]['label'].values[0])
#         ambiguous_mixup_data['category_2'] = ambiguous_mixup_data['idx_2'].apply(lambda x: ambiguous_data[ambiguous_data['idx'] == x]['category'].values[0])
#         ambiguous_mixup_data['mixup_type'] = 'same_ambiguous'
        
#         same_mixup_data = pd.concat([easy_mixup_data, ambiguous_mixup_data])
        
#         # --------------------------------------- Different class mixup ---------------------------------------  
        
#         # Random easy-ambi mixup
#         different_samples = mixup_size - len(same_mixup_data)
#         easy_tuple = list(zip(easy_data['idx'].tolist(), easy_data['text'].tolist(), easy_data['label'].tolist(), easy_data['category'].tolist()))
#         ambiguous_tuple = list(zip(ambiguous_data['idx'].tolist(), ambiguous_data['text'].tolist(), ambiguous_data['label'].tolist(), ambiguous_data['category'].tolist()))
        
#         easy_data = easy_data.sample(n=different_samples//2).reset_index(drop=True)
#         ambiguous4easy = random.choices(ambiguous_tuple, weights=np.ones(len(ambiguous_tuple)), k=different_samples//2)
#         ambiguous4easy = pd.DataFrame(ambiguous4easy, columns=['idx_2', 'text_2', 'label_2', 'category_2'])
#         ambiguous4easy = pd.concat([easy_data, ambiguous4easy], axis=1).reset_index(drop=True)
#         ambiguous4easy['mixup_type'] = 'ambiguous_easy'
        
#         ambiguous_data = ambiguous_data.sample(n=different_samples//2).reset_index(drop=True)
#         easy4ambiguous = random.choices(easy_tuple, weights=np.ones(len(easy_tuple)), k=different_samples//2)
#         easy4ambiguous = pd.DataFrame(easy4ambiguous, columns=['idx_2', 'text_2', 'label_2', 'category_2'])
#         easy4ambiguous = pd.concat([ambiguous_data, easy4ambiguous], axis=1).reset_index(drop=True)
#         easy4ambiguous['mixup_type'] = 'easy_ambiguous'
        
#         return pd.concat([same_mixup_data, easy4ambiguous, ambiguous4easy]).sample(frac=1).reset_index(drop=True)

#     return data

In [6]:
def get_label_data(df, label=0):
    df_label = df[df['label'] == label].reset_index(drop=True)
    temp_label = df_label.sample(frac=1).reset_index(drop=True)
    temp_label = temp_label.rename(columns={'idx': 'idx_2', 'text': 'text_2', 'label': 'label_2', 'category': 'category_2'})   
    return pd.concat([df_label, temp_label], axis=1).reset_index(drop=True)

In [20]:
def prepare_dataset_original(data):
#     return data[(data['category'] != 'none') & (data['category'] != 'hard')].sample(frac=1).reset_index(drop=True)
    return data[data['category'] != 'hard'].reset_index(drop=True)

In [21]:
def prepare_dataset_random_mixup(data, info_data=None, use_label=False):
    data_len = len(data)
    data = data[(data['category'] != 'none') & (data['category'] != 'hard')].reset_index(drop=True)
    
    if use_label:
        data_0 = get_label_data(data, label=0)
        data_1 = get_label_data(data, label=1)
        final_data = pd.concat([data_0, data_1]).reset_index(drop=True)
        
    else:
        temp = data.copy()
        temp = temp.sample(frac=1).reset_index(drop=True)
        temp = temp.rename(columns={'idx': 'idx_2', 'text': 'text_2', 'label': 'label_2', 'category': 'category_2'})
        final_data = pd.concat([data, temp], axis=1)
        
    random_subset_1 = data.sample(n=data_len-len(final_data)).reset_index(drop=True)
    random_subset_2 = data.sample(n=data_len-len(final_data)).reset_index(drop=True)
    random_subset_2 = random_subset_2.rename(columns={'idx': 'idx_2', 'text': 'text_2', 'label': 'label_2', 'category': 'category_2'})
    random_subset = pd.concat([random_subset_1, random_subset_2], axis=1).reset_index(drop=True)  

    return pd.concat([final_data, random_subset]).sample(frac=1).reset_index(drop=True)

In [22]:
def prepare_dataset_category_mixup(data, info_data=None, use_label=False, use_entropy=False):
    data_len = len(data)
    data = data[(data['category'] != 'none') & (data['category'] != 'hard')].reset_index(drop=True)
#     data = data[data['category'] != 'hard'].reset_index(drop=True)
    
    if use_entropy:
        # Easy-Easy Mixup
        easy_data = data[data['category'] == 'easy']
        easy_low_ent_idx = easy_data.sort_values('entropy', ascending=True).head(mixup_size//2)['idx'].tolist()
        easy_high_ent_idx = easy_data.sort_values('entropy', ascending=False).head(mixup_size//2)['idx'].tolist()
        
        easy_mixup_data = easy_data[easy_data['idx'].isin(easy_low_ent_idx)].reset_index(drop=True)
        random.shuffle(easy_high_ent_idx)
        easy_mixup_data['idx_2'] = easy_high_ent_idx
        easy_mixup_data['text_2'] = easy_mixup_data['idx_2'].apply(lambda x: easy_data[easy_data['idx'] == x]['text'].values[0])
        easy_mixup_data['label_2'] = easy_mixup_data['idx_2'].apply(lambda x: easy_data[easy_data['idx'] == x]['label'].values[0])
        easy_mixup_data['category_2'] = easy_mixup_data['idx_2'].apply(lambda x: easy_data[easy_data['idx'] == x]['category'].values[0])
        easy_mixup_data['mixup_type'] = 'same_easy'
        
        # Ambi-Ambi Mixup
        ambiguous_data = data[data['category'] == 'ambiguous']
        ambiguous_low_ent_idx = ambiguous_data.sort_values('entropy', ascending=True).head(mixup_size//2)['idx'].tolist()
        ambiguous_high_ent_idx = ambiguous_data.sort_values('entropy', ascending=False).head(mixup_size//2)['idx'].tolist()
        
        ambiguous_mixup_data = ambiguous_data[ambiguous_data['idx'].isin(ambiguous_low_ent_idx)].reset_index(drop=True)
        random.shuffle(ambiguous_high_ent_idx)
        ambiguous_mixup_data['idx_2'] = ambiguous_high_ent_idx
        ambiguous_mixup_data['text_2'] = ambiguous_mixup_data['idx_2'].apply(lambda x: ambiguous_data[ambiguous_data['idx'] == x]['text'].values[0])
        ambiguous_mixup_data['label_2'] = ambiguous_mixup_data['idx_2'].apply(lambda x: ambiguous_data[ambiguous_data['idx'] == x]['label'].values[0])
        ambiguous_mixup_data['category_2'] = ambiguous_mixup_data['idx_2'].apply(lambda x: ambiguous_data[ambiguous_data['idx'] == x]['category'].values[0])
        ambiguous_mixup_data['mixup_type'] = 'same_ambiguous'
        
        final_data = pd.concat([easy_mixup_data, ambiguous_mixup_data]).sample(frac=1).reset_index(drop=True)
    
    else:
        if use_label:
            # Easy-Easy mixup
            easy_data = data[data['category'] == 'easy'].reset_index(drop=True)
            easy_data_0 = get_label_data(easy_data, label=0)
            easy_data_1 = get_label_data(easy_data, label=1)
            easy_data  = pd.concat([easy_data_0, easy_data_1]).reset_index(drop=True)
            
            # Ambi-Ambi mixup
            ambiguous_data = data[data['category'] == 'ambiguous'].reset_index(drop=True)
            ambiguous_data_0 = get_label_data(ambiguous_data, label=0)
            ambiguous_data_1 = get_label_data(ambiguous_data, label=1)
            ambiguous_data  = pd.concat([ambiguous_data_0, ambiguous_data_1]).reset_index(drop=True)
            
            final_data = pd.concat([easy_data, ambiguous_data]).sample(frac=1).reset_index(drop=True)
            
            # Easy-Ambi Mixup
            
            
        else:
            # Easy-Easy mixup
            easy_data = data[data['category'] == 'easy'].reset_index(drop=True)
            easy_temp = easy_data.copy()
            easy_temp = easy_temp.sample(frac=1).reset_index(drop=True)
            easy_temp = easy_temp.rename(columns={'idx': 'idx_2', 'text': 'text_2', 'label': 'label_2', 'category': 'category_2'})
            easy_data = pd.concat([easy_data, easy_temp], axis=1).reset_index(drop=True)
            easy_data['mixup_type'] = 'same_easy'

            # Ambi-Ambi mixup
            ambiguous_data = data[data['category'] == 'ambiguous'].reset_index(drop=True)
            ambiguous_temp = ambiguous_data.copy()
            ambiguous_temp = ambiguous_temp.sample(frac=1).reset_index(drop=True)
            ambiguous_temp = ambiguous_temp.rename(columns={'idx': 'idx_2', 'text': 'text_2', 'label': 'label_2', 'category': 'category_2'})
            ambiguous_data = pd.concat([ambiguous_data, ambiguous_temp], axis=1).reset_index(drop=True)
            ambiguous_data['mixup_type'] = 'same_ambiguous'
            
            final_data = pd.concat([easy_data, ambiguous_data]).sample(frac=1).reset_index(drop=True)
            
            # Easy-Ambi Mixup
            easy_ambiguous_len = data_len - len(final_data)
            easy_ambiguous_data = easy_data.sample(n=easy_ambiguous_len)[['idx', 'text', 'label', 'category']].reset_index(drop=True)
            easy_ambiguous_temp = ambiguous_data.sample(n=easy_ambiguous_len)[['idx', 'text', 'label', 'category']].reset_index(drop=True)
            easy_ambiguous_temp = easy_ambiguous_temp.rename(columns={'idx': 'idx_2', 'text': 'text_2', 'label': 'label_2', 'category': 'category_2'})
            easy_ambiguous_data = pd.concat([easy_ambiguous_data, easy_ambiguous_temp], axis=1)
            
            return pd.concat([final_data, easy_ambiguous_data]).sample(frac=1).reset_index(drop=True)

In [23]:
# def prepare_dataset(data, info_data=None, mixup=False):
    
#     # Remove none and hard examples
#     data = data[(data['category'] != 'none') & (data['category'] != 'hard')].reset_index(drop=True)

#     # Add softmax and entropy info
#     if (info_data is not None) and mixup:
#         data = pd.merge(data, info_data, on='idx')[['idx', 'text', 'label', 'category', 'softmax', 'entropy']]
#         mixup_size = len(info_data) - len(data)

#         # --------------------------------------- Same class mixup ---------------------------------------  

#         # Easy-Easy Mixup
#         easy_data = data[data['category'] == 'easy']
#         easy_low_ent_idx = easy_data.sort_values('entropy', ascending=True).head(mixup_size//2)['idx'].tolist()
#         easy_high_ent_idx = easy_data.sort_values('entropy', ascending=False).head(mixup_size//2)['idx'].tolist()
        
#         easy_mixup_data = easy_data[easy_data['idx'].isin(easy_low_ent_idx)].reset_index(drop=True)
#         random.shuffle(easy_high_ent_idx)
#         easy_mixup_data['idx_2'] = easy_high_ent_idx
#         easy_mixup_data['text_2'] = easy_mixup_data['idx_2'].apply(lambda x: easy_data[easy_data['idx'] == x]['text'].values[0])
#         easy_mixup_data['label_2'] = easy_mixup_data['idx_2'].apply(lambda x: easy_data[easy_data['idx'] == x]['label'].values[0])
#         easy_mixup_data['category_2'] = easy_mixup_data['idx_2'].apply(lambda x: easy_data[easy_data['idx'] == x]['category'].values[0])
#         easy_mixup_data['mixup_type'] = 'same_easy'
        
#         # Ambi-Ambi Mixup
#         ambiguous_data = data[data['category'] == 'ambiguous']
#         ambiguous_low_ent_idx = ambiguous_data.sort_values('entropy', ascending=True).head(mixup_size//2)['idx'].tolist()
#         ambiguous_high_ent_idx = ambiguous_data.sort_values('entropy', ascending=False).head(mixup_size//2)['idx'].tolist()
        
#         ambiguous_mixup_data = ambiguous_data[ambiguous_data['idx'].isin(ambiguous_low_ent_idx)].reset_index(drop=True)
#         random.shuffle(ambiguous_high_ent_idx)
#         ambiguous_mixup_data['idx_2'] = ambiguous_high_ent_idx
#         ambiguous_mixup_data['text_2'] = ambiguous_mixup_data['idx_2'].apply(lambda x: ambiguous_data[ambiguous_data['idx'] == x]['text'].values[0])
#         ambiguous_mixup_data['label_2'] = ambiguous_mixup_data['idx_2'].apply(lambda x: ambiguous_data[ambiguous_data['idx'] == x]['label'].values[0])
#         ambiguous_mixup_data['category_2'] = ambiguous_mixup_data['idx_2'].apply(lambda x: ambiguous_data[ambiguous_data['idx'] == x]['category'].values[0])
#         ambiguous_mixup_data['mixup_type'] = 'same_ambiguous'
        
#         return pd.concat([easy_mixup_data, ambiguous_mixup_data]).sample(frac=1).reset_index(drop=True)
    
#     return data

In [24]:
def get_count(df, label1, label2):
    cnt_label = 0
    cnt_category = 0
    for i in range(len(df)):
        if df.at[i, label1] == df.at[i, f"{label1}_2"]:
            cnt_label += 1
        if df.at[i, label2] == df.at[i, f"{label2}_2"]:
            cnt_category += 1

    print("same label: ", cnt_label/len(df))
    print("same category: ", cnt_category/len(df))

In [25]:
df = pd.read_csv('/projects/metis2/atharvak/Data_Cartography/datasets/imdb/imdb_categorized.csv')
df

,idx,text,label,category
0,0,I rented I AM CURIOUS-YELLOW from my video sto...,0,ambiguous
1,1,"""I Am Curious: Yellow"" is a risible and preten...",0,easy
2,2,If only to avoid making this type of film in t...,0,easy
3,3,This film was probably inspired by Godard's Ma...,0,easy
4,4,"Oh, brother...after hearing about this ridicul...",0,none
...,...,...,...,...
24995,24995,A hit at the time but now better categorised a...,1,ambiguous
24996,24996,I love this movie like no other. Another time ...,1,none
24997,24997,This film and it's sequel Barry Mckenzie holds...,1,none
24998,24998,'The Adventures Of Barry McKenzie' started lif...,1,ambiguous


In [26]:
info_df = pd.read_csv('/projects/metis2/atharvak/Data_Cartography/dy_log/imdb/roberta-base/training_dynamics/final_4.csv')
info_df = info_df.rename(columns={'guid': 'idx', 'sm': 'softmax', 'en': 'entropy'})
info_df

,Unnamed: 0,idx,gold,softmax,entropy
0,0,23493,1,[8.37615955e-04 9.99162384e-01],0.006772
1,1,24825,1,[0.0032071 0.9967929],0.021618
2,2,20509,1,[0.00222997 0.99777003],0.015843
3,3,10239,0,[9.99423855e-01 5.76144926e-04],0.004874
4,4,10998,0,[9.99719573e-01 2.80426921e-04],0.002574
...,...,...,...,...,...
24995,24995,4317,0,[0.99804368 0.00195632],0.014155
24996,24996,23824,1,[0.00474844 0.99525156],0.030141
24997,24997,332,0,[9.99438949e-01 5.61051043e-04],0.004761
24998,24998,11183,0,[0.96347178 0.03652822],0.156749


In [27]:
normal_dataset = prepare_dataset_original(df)
normal_dataset

,idx,text,label,category
0,0,I rented I AM CURIOUS-YELLOW from my video sto...,0,ambiguous
1,1,"""I Am Curious: Yellow"" is a risible and preten...",0,easy
2,2,If only to avoid making this type of film in t...,0,easy
3,3,This film was probably inspired by Godard's Ma...,0,easy
4,4,"Oh, brother...after hearing about this ridicul...",0,none
...,...,...,...,...
24537,24995,A hit at the time but now better categorised a...,1,ambiguous
24538,24996,I love this movie like no other. Another time ...,1,none
24539,24997,This film and it's sequel Barry Mckenzie holds...,1,none
24540,24998,'The Adventures Of Barry McKenzie' started lif...,1,ambiguous


In [15]:
random_mixup_dataset = prepare_dataset_random_mixup(df, info_data=info_df, use_label=False)
get_count(random_mixup_dataset, 'label', 'category')
random_mixup_dataset

same label:  0.50736
same category:  0.49936


,idx,text,label,category,idx_2,text_2,label_2,category_2
0,2102,Quite frankly it seemed like seven hours of bo...,0,easy,28,Some films that you pick up for a pound turn o...,0,easy
1,600,I've been going through the AFI's list of the ...,0,easy,13182,"Fascist principal Miss Togar(Mary Woronov, who...",1,ambiguous
2,7403,"When I saw the previews for this movie, I didn...",0,easy,24303,Ya know when one looks at this Brian DePalma f...,1,ambiguous
3,18873,Great fun. I went with 8 friends to a sneak pr...,1,ambiguous,8080,The mood of this movie is pretty good and it c...,0,ambiguous
4,22667,"A strong woman oriented subject after long, di...",1,ambiguous,16542,A complex story laid on the background of part...,1,ambiguous
...,...,...,...,...,...,...,...,...
24995,3287,This is the first Guinea Pig film from Japan a...,0,easy,12808,This movie of 370 minutes was aired by the Ita...,1,ambiguous
24996,4254,This was the WORST Christmas movie I ever saw....,0,easy,11730,"A fun concept, but poorly executed. Except for...",0,easy
24997,3674,I hated this show when I was a kid. That was b...,0,ambiguous,18066,Cheech & Chong's Next Movie (1980) was the sec...,1,ambiguous
24998,4887,"I was expecting the movie based on Grendel, th...",0,easy,18086,The silent film the Pride of the Clan starring...,1,ambiguous


In [26]:
random_mixup_dataset = prepare_dataset_random_mixup(df, info_data=info_df, use_label=True)
get_count(random_mixup_dataset, 'label', 'category')
random_mixup_dataset

same label:  0.82956
same category:  0.53616


,idx,text,label,category,idx_2,text_2,label_2,category_2
0,7731,I remember watching ATTACK when it first came ...,0,easy,5961,If this is supposed to be the black experience...,0,ambiguous
1,17744,"An unmarried, twenty-something hick (played by...",1,ambiguous,21644,Playing a character from a literary classic ca...,1,ambiguous
2,13968,"He's stocky, sweaty, slightly cross-eyed and r...",1,ambiguous,15257,Imagine turning out the lights in your remote ...,1,ambiguous
3,21265,"I meant that in a GOOD way, believe me. True t...",1,ambiguous,15926,While Rome goes mad celebrating Hitler's visit...,1,ambiguous
4,15037,"Cinematography--Compared to 'The Wrestler,' a ...",1,ambiguous,5021,<br /><br />Whether any indictment was intende...,0,ambiguous
...,...,...,...,...,...,...,...,...
24995,23329,I'm on the opposite end of the previous commen...,1,ambiguous,19209,"I used to write comments at IMDb, but I don't ...",1,ambiguous
24996,345,SHALLOW GRAVE begins with either a tribute or ...,0,ambiguous,7298,"Watching beautiful women sneaking around, play...",0,ambiguous
24997,12900,<br /><br />There is STAR TREK canon -- lots o...,1,ambiguous,23936,Zero day has a purpose and this is not simply ...,1,ambiguous
24998,5134,Pretty awful but watchable and entertaining. I...,0,ambiguous,4324,1 out of 10.<br /><br />This is the kind of mo...,0,easy


In [27]:
category_mixup_dataset = prepare_dataset_category_mixup(df, info_data=info_df, use_label=False, use_entropy=False)
get_count(category_mixup_dataset, 'label', 'category')
category_mixup_dataset

ValueError: Cannot take a larger sample than population when 'replace=False'

In [18]:
category_mixup_dataset = prepare_dataset_category_mixup(df, info_data=info_df, use_label=True, use_entropy=False)
get_count(category_mixup_dataset, 'label', 'category')
category_mixup_dataset

same label:  0.82976
same category:  0.83052


,idx,text,label,category,idx_2,text_2,label_2,category_2
0,9662,Interesting? Hardly. The 'scientific evidence'...,0,easy,5958,A black guy fights ..... and supposedly wins ....,0,easy
1,7507,Probably the only thing that got the movie up ...,0,ambiguous,5482,I think this is one of the weakest of the Kenn...,0,ambiguous
2,16012,Scott Henderson (Alan Curtis) meets a mystery ...,1,ambiguous,7759,....this mini does not get better with age. I ...,0,easy
3,18021,It took a long time until I could find the tit...,1,ambiguous,15581,This is your standard musical comedy from the ...,1,ambiguous
4,8846,"I must admit, there are few books with corresp...",0,easy,10887,Words cannot describe how utterly abysmal this...,0,easy
...,...,...,...,...,...,...,...,...
24995,8012,"I'm sure this was one of those ""WOAH!"" attract...",0,easy,20496,This movie is great entertainment to watch wit...,1,easy
24996,16559,'The Mother' is that extraordinary piece of fi...,1,easy,23995,An excellent family movie... gives a lot to th...,1,easy
24997,14413,Saw this Saturday night at the Provincetown Fi...,1,easy,19821,Xizao is a rare little movie. It is simple and...,1,easy
24998,19143,"To be honest, I had to go see this movie backw...",1,ambiguous,10197,how can this movie have a 5.5 this movie was a...,0,easy


In [9]:
mixup_dataset['mixup_type'].value_counts()

same_easy         2833
same_ambiguous    2833
easy_ambiguous    1417
ambiguous_easy    1417
Name: mixup_type, dtype: int64

In [11]:
mixup_dataset['unique_pair'] = mixup_dataset.apply(lambda x: "_".join(str(index) for index in sorted([x.idx, x.idx_2])), axis=1)
mixup_dataset

,idx,text,label,category,sm,en,idx_2,label_2,category_2,mixup_type,unique_pair
0,17484,It takes a Serbian or at least a Balkan famili...,1,ambiguous,[9.55682524e-04 9.99044317e-01],0.007600,16986,1,ambiguous,same,16986_17484
1,7166,"What else can you say about this movie,except ...",0,easy,[9.99527225e-01 4.72774999e-04],0.004093,2838,0,ambiguous,different,2838_7166
2,16389,"Passionate, dramatic, riveting as Flamenco its...",1,easy,[0.00268855 0.99731145],0.018598,1789,0,easy,same,1789_16389
3,1891,Such is the dilemma(above) that Debbie must fa...,0,ambiguous,[0.76936242 0.23063758],0.540046,1381,0,ambiguous,same,1381_1891
4,18374,Since I am so interested in lake monsters i re...,1,ambiguous,[0.0023722 0.9976278],0.016707,10354,0,easy,different,10354_18374
...,...,...,...,...,...,...,...,...,...,...,...
32995,20219,This gloriously turgid melodrama represents Do...,1,ambiguous,[7.48858525e-04 9.99251141e-01],0.006138,2190,0,ambiguous,same,2190_20219
32996,8508,I was so disappointed by this show. After hear...,0,easy,[9.99726178e-01 2.73821708e-04],0.002520,6912,0,easy,same,6912_8508
32997,133,Having seen three other versions of the same f...,0,ambiguous,[9.99084106e-01 9.15893645e-04],0.007323,9262,0,easy,different,133_9262
32998,21351,The autobiography on which this movie is based...,1,easy,[0.00133933 0.99866067],0.010199,8020,0,easy,same,8020_21351


In [9]:
mixup_dataset.drop_duplicates(subset=["unique_pair"])

,idx,text,label,category,idx_2,label_2,category_2,mixup_type,unique_pair
0,24929,"Well, this film is a difficult one really. To ...",1,ambiguous,10558,0,ambiguous,same,10558_24929
1,17556,I didn't expect much when I first saw the DVD ...,1,easy,1515,0,ambiguous,different,1515_17556
2,8880,This is one of those horror flicks where twent...,0,ambiguous,8804,0,easy,different,8804_8880
3,513,The Good Earth is perhaps the most boring film...,0,easy,21561,1,ambiguous,different,513_21561
4,22906,Forget the campy 'religious' movies that have ...,1,ambiguous,5824,0,easy,different,5824_22906
...,...,...,...,...,...,...,...,...,...
32995,2378,Let's hope this is the final nightmare. This i...,0,easy,12304,0,ambiguous,different,2378_12304
32996,22330,...in our household. Like everyone else who ha...,1,ambiguous,10697,0,easy,different,10697_22330
32997,15119,I think Lion King 1 1/2 is one of the best seq...,1,easy,5661,0,easy,same,5661_15119
32998,24347,"Time for a rant, eh: I thought Spirit was a gr...",1,ambiguous,11618,0,easy,different,11618_24347


In [7]:
class MixupDataset(Dataset):
    
    def __init__(
        self, 
        tokenizer, 
        data: pd.DataFrame,
        sampling_type: str
    ):
        self.data = data
        self.sampling_type = sampling_type
        
        if self.sampling_type == 'sequential':
            sorting_dict = {
                'non_mixup_easy': 0,
                'non_mixup_ambiguous': 1,
                'same_easy': 2,
                'different_easy': 3,
                'same_ambiguous': 4,
                'different_ambiguous': 5
            }
            self.data['data_type'] = self.data['mixup_type'] + '_' + self.data['category']
            self.data = self.data.iloc[self.data.data_type.map(sorting_dict).argsort()].reset_index(drop=True)
            
        self.tokenizer = tokenizer
        self.tokenized_data = tokenizer.batch_encode_plus(
            self.data[INPUT_COLUMN].tolist(),
            max_length=MAX_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_token_type_ids=True,            
            return_tensors='pt'
        )
        

                
    def __len__(
        self
    ):
        return len(self.data)
    
    
    
    def __getitem__(
        self,
        index: int
    ):
        data = {
            'input_ids_1': self.tokenized_data['input_ids'][index].flatten(),
            'attention_mask_1': self.tokenized_data['attention_mask'][index].flatten(),
            'labels_1': torch.tensor(self.data.iloc[index][OUTPUT_COLUMN], dtype=torch.long),
        }
        
        idx2 = self.data.iloc[index]['idx_2']

        index2 = self.data[self.data['idx'] == idx2].index[0]
        data['input_ids_2'] = self.tokenized_data['input_ids'][index2].flatten()
        data['attention_mask_2'] = self.tokenized_data['attention_mask'][index2].flatten()
        data['labels_2'] = torch.tensor(self.data.iloc[index2][OUTPUT_COLUMN], dtype=torch.long)

        return data

In [8]:
train_dataset = MixupDataset(tokenizer=tokenizer, data=dataset, sampling_type='random')
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=False)

## Training loop

In [6]:
def train(model, tokenizer, optimizer, device, train_loader, num_epochs, output_dir):
    losses = []
    train_iterator = trange(int(num_epochs), desc='Epoch')
    for _ in train_iterator:
        tr_loss = 0
        step = None
        epoch_iterator = tqdm(train_loader, desc='Training')
        for step, batch in enumerate(epoch_iterator):
            model.train()

            inputs = {}
            for k, v in batch.items():
                if isinstance(v, list):
                    inputs[k] = None
                else:
                    inputs[k] = v.to(device)

            labels = inputs['labels_1']

            optimizer.zero_grad()

            outputs = model(**inputs)
            out = outputs['logits'].double().to(device)
            loss = outputs['loss']

            loss.backward()
            optimizer.step()
    
            tr_loss += loss.item()
        losses.append(tr_loss/(step+1))
        print('train loss: {}'.format(tr_loss/(step+1)))

    # save model and tokenizer
    print('Saving model and tokenizer')

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)


In [7]:
def eval(model, eval_loader, device, with_labels=True):
    probs = None
    gold_labels = None

    eval_loss = 0
    step = None
    eval_iterator = tqdm(eval_loader, desc='Evaluating')
    for step, batch in enumerate(eval_iterator):
        model.eval()

        with torch.no_grad():
            
            inputs = {k:v.to(device) for k, v in batch.items()}
            labels = inputs['labels_1']
            del inputs['labels_1']
            del inputs['labels_2']
 
            outputs = model(**inputs)

            out = outputs['logits'].double().to(device)
            out = F.softmax(out, dim=1)

            loss = outputs['loss']

            if probs is None:
                probs = out.detach().cpu().numpy()
                if with_labels:
                    gold_labels = labels.detach().cpu().numpy()
            else:
                probs = np.append(probs, out.detach().cpu().numpy(), axis=0)
                if with_labels:
                    gold_labels = np.append(gold_labels, labels.detach().cpu().numpy(), axis=0)

            if with_labels:
                eval_loss += loss.item()
    
    if with_labels:
        eval_loss /= (step+1)
        print('eval loss: {}'.format(eval_loss))

        # compute accuracy
        preds = np.argmax(probs, axis=1)
        accuracy = np.sum(preds == gold_labels)/len(preds)
        print('eval accuracy: {}'.format(accuracy))

    return probs

In [1]:
import json
import pandas as pd

In [2]:
with open('../dy_log/imdb/roberta-base/three_regions_data_indices.json', 'r') as f:
    data_imdb = json.load(f)
f.close()
print(data_imdb.keys())

df_imdb = pd.read_csv('/projects/ogma3/atharvak/Data_Cartography/datasets/imdb/train.csv')
len(df_imdb), len(data_imdb['easy']), len(data_imdb['ambiguous']), len(data_imdb['hard']), len(data_imdb['easy'])/len(df_imdb)

dict_keys(['hard', 'easy', 'ambiguous'])


(25000, 8250, 8250, 8250, 0.33)

In [6]:
set(data_imdb['easy']).intersection(set(data_imdb['ambiguous'])), set(data_imdb['easy']).intersection(set(data_imdb['hard'])), set(data_imdb['hard']).intersection(set(data_imdb['ambiguous']))

(set(),
 set(),
 {0,
  13,
  14,
  20,
  23,
  32,
  39,
  48,
  50,
  52,
  53,
  55,
  61,
  65,
  69,
  70,
  74,
  77,
  79,
  80,
  81,
  84,
  87,
  89,
  90,
  92,
  97,
  98,
  105,
  108,
  109,
  122,
  123,
  124,
  130,
  132,
  133,
  136,
  138,
  142,
  156,
  157,
  158,
  163,
  167,
  168,
  171,
  173,
  179,
  180,
  188,
  190,
  198,
  199,
  200,
  224,
  228,
  233,
  236,
  245,
  246,
  252,
  255,
  257,
  260,
  263,
  264,
  267,
  271,
  278,
  279,
  280,
  281,
  285,
  289,
  292,
  299,
  301,
  304,
  306,
  313,
  318,
  336,
  339,
  340,
  342,
  345,
  346,
  348,
  354,
  362,
  366,
  367,
  371,
  373,
  375,
  377,
  378,
  384,
  387,
  394,
  397,
  403,
  405,
  406,
  408,
  410,
  413,
  418,
  419,
  421,
  427,
  430,
  440,
  441,
  446,
  455,
  457,
  459,
  461,
  464,
  469,
  471,
  474,
  475,
  476,
  481,
  483,
  488,
  492,
  497,
  498,
  499,
  501,
  504,
  509,
  511,
  515,
  516,
  518,
  522,
  526,
  527,
  528,
  532

In [3]:
with open('../dy_log/sst2/roberta-base/three_regions_data_indices.json', 'r') as f:
    data_sst2 = json.load(f)
f.close()
print(data_sst2.keys())
df_sst2 = pd.read_csv('/projects/ogma3/atharvak/Data_Cartography/datasets/sst2/train.csv')
len(df_sst2), len(data_sst2['easy']), len(data_sst2['ambiguous']), len(data_sst2['hard']), len(data_sst2['easy'])/len(df_sst2)

dict_keys(['hard', 'easy', 'ambiguous'])


(67349, 22225, 22225, 22225, 0.3299974758348305)

In [4]:
with open('../dy_log/yelp_polarity/roberta-base/three_regions_data_indices.json', 'r') as f:
    data_yelp_polarity = json.load(f)
f.close()
print(data_yelp_polarity.keys())
df_yelp_polarity = pd.read_csv('/projects/ogma3/atharvak/Data_Cartography/datasets/yelp_polarity/train.csv')
len(df_yelp_polarity), len(data_yelp_polarity['easy']), len(data_yelp_polarity['ambiguous']), len(data_yelp_polarity['hard']), len(data_yelp_polarity['easy'])/len(df_yelp_polarity)

dict_keys(['hard', 'easy', 'ambiguous'])


(560000, 184800, 184800, 184800, 0.33)